In [79]:
import numpy as np
import csv
import pandas as pd
import requests
import json
import time
import yfinance as yf
from datetime import datetime,timedelta

In [93]:
# https://xbrl.fasb.org/us-gaap/2022/elts/us-gaap-2022.xsd
headers = {
        'User-Agent': 'River Holdings river@gmail.com',
        'Accept-Encoding': 'gzip, deflate',
        'Host': 'data.sec.gov',
    }
response={}
year_range=range(2009,2023)
all_gaap_elements=['NetIncomeLoss','Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax']

for year in year_range:
    for index, element in enumerate(all_gaap_elements):
        URL = f"https://data.sec.gov/api/xbrl/frames/us-gaap/{all_gaap_elements[index]}/USD/CY{year}.json"
        response[year,element]= requests.get(URL, headers=headers).text
        response[year,element]= response[year,element][response[year,element].find('['):response[year,element].rfind(']')+1]
        print(year, element, len(response[year,element]))
        time.sleep(.1)


2008 NetIncomeLoss 292084
2008 Revenues 97782
2008 RevenueFromContractWithCustomerExcludingAssessedTax 0
2009 NetIncomeLoss 701681
2009 Revenues 278445
2009 RevenueFromContractWithCustomerExcludingAssessedTax 0
2010 NetIncomeLoss 1065909
2010 Revenues 473121
2010 RevenueFromContractWithCustomerExcludingAssessedTax 0
2011 NetIncomeLoss 1187682
2011 Revenues 588154
2011 RevenueFromContractWithCustomerExcludingAssessedTax 0
2012 NetIncomeLoss 1158660
2012 Revenues 591706
2012 RevenueFromContractWithCustomerExcludingAssessedTax 0
2013 NetIncomeLoss 1130473
2013 Revenues 572471
2013 RevenueFromContractWithCustomerExcludingAssessedTax 0
2014 NetIncomeLoss 1094172
2014 Revenues 549824
2014 RevenueFromContractWithCustomerExcludingAssessedTax 168
2015 NetIncomeLoss 1036009
2015 Revenues 515634
2015 RevenueFromContractWithCustomerExcludingAssessedTax 4948
2016 NetIncomeLoss 986194
2016 Revenues 554676
2016 RevenueFromContractWithCustomerExcludingAssessedTax 216089
2017 NetIncomeLoss 955258
2017 

In [81]:
df_dict={}

for year in year_range:
    for index, element in enumerate(all_gaap_elements):
        print(year, element)
        if len(response[year,element]):
            json_object = json.loads(response[year,element])
            df_dict[year,element] = pd.DataFrame.from_dict(json_object)
            df_dict[year,element] = df_dict[year,element].drop(['accn','loc', 'start'], axis='columns')
            df_dict[year,element]=df_dict[year,element].rename(columns={'end':'date', 'val':f'{element}'})



2009 NetIncomeLoss
2009 Revenues
2009 RevenueFromContractWithCustomerExcludingAssessedTax
2010 NetIncomeLoss
2010 Revenues
2010 RevenueFromContractWithCustomerExcludingAssessedTax
2011 NetIncomeLoss
2011 Revenues
2011 RevenueFromContractWithCustomerExcludingAssessedTax
2012 NetIncomeLoss
2012 Revenues
2012 RevenueFromContractWithCustomerExcludingAssessedTax
2013 NetIncomeLoss
2013 Revenues
2013 RevenueFromContractWithCustomerExcludingAssessedTax
2014 NetIncomeLoss
2014 Revenues
2014 RevenueFromContractWithCustomerExcludingAssessedTax
2015 NetIncomeLoss
2015 Revenues
2015 RevenueFromContractWithCustomerExcludingAssessedTax
2016 NetIncomeLoss
2016 Revenues
2016 RevenueFromContractWithCustomerExcludingAssessedTax
2017 NetIncomeLoss
2017 Revenues
2017 RevenueFromContractWithCustomerExcludingAssessedTax
2018 NetIncomeLoss
2018 Revenues
2018 RevenueFromContractWithCustomerExcludingAssessedTax
2019 NetIncomeLoss
2019 Revenues
2019 RevenueFromContractWithCustomerExcludingAssessedTax
2020 NetIn

In [82]:
df=pd.DataFrame()

for index, element in enumerate(all_gaap_elements):
    df_dict[element]= pd.DataFrame()
    for year in year_range:
        try:
            df_dict[element]= pd.concat([df_dict[element],df_dict[year,element]])
        except Exception as e:
            print(e)

for index, element in enumerate(all_gaap_elements):
    if index==0:
        df=df_dict[element]
    else:
        try:
            df=df.join(df_dict[element].set_index(['cik','entityName','date']), on=['cik','entityName','date'])
        except:
            print('Your year range is likely missing an element')
    
df=df.dropna(subset=['Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax'],how='all')
df['Revenues']= np.where(~df['Revenues'].isnull(), df['Revenues'],df['RevenueFromContractWithCustomerExcludingAssessedTax'])
df=df.drop('RevenueFromContractWithCustomerExcludingAssessedTax', axis='columns')
df

(2009, 'RevenueFromContractWithCustomerExcludingAssessedTax')
(2010, 'RevenueFromContractWithCustomerExcludingAssessedTax')
(2011, 'RevenueFromContractWithCustomerExcludingAssessedTax')
(2012, 'RevenueFromContractWithCustomerExcludingAssessedTax')
(2013, 'RevenueFromContractWithCustomerExcludingAssessedTax')


,cik,entityName,date,NetIncomeLoss,Revenues
0,864328,BJ SERVICES CO,2009-09-30,1.499430e+08,4.121897e+09
3,64978,MERCK SHARP & DOHME CORP.,2009-12-31,7.952300e+09,2.364320e+10
4,1335793,CNX GAS CORP,2009-12-31,1.644620e+08,6.834440e+08
5,868809,XTO ENERGY INC,2009-12-31,2.019000e+09,9.064000e+09
8,67099,MITSUI & CO LTD,2010-03-31,1.610000e+09,4.404800e+10
...,...,...,...,...,...
96,1786352,"BILL.COM HOLDINGS, INC.",2022-06-30,-3.263610e+08,6.419590e+08
97,1794783,"SelectQuote, Inc.",2022-06-30,-2.975040e+08,7.640450e+08
98,1795250,MADISON SQUARE GARDEN ENTERTAINMENT CORP.,2022-06-30,-1.943950e+08,1.724618e+09
101,1820953,"Affirm Holdings, Inc.",2022-06-30,-7.074170e+08,1.349292e+09


In [83]:
revenue_limit=100000000
size_before=len(df)
df=df[df['Revenues']>revenue_limit]
print(f'Length Before {size_before} Length after {len(df)}')

Length Before 48769 Length after 26213


In [84]:
path = '/Users/rweiss/Documents/Yfinance Code/EDGAR Fundamentals/company_tickers_to_cik.json'
with open(path) as train_file:
    json_dict = json.load(train_file)
df_tickers= pd.DataFrame.from_dict(json_dict).T
df_tickers=df_tickers.rename(columns={'cik_str':'cik'})
df_tickers=df_tickers.drop(['title'], axis='columns')

df = pd.merge(df,df_tickers, on='cik')
df.rename(str.lower, axis='columns')
df=df.sort_values(by=['date']).reset_index(drop=True)

df


,cik,entityName,date,NetIncomeLoss,Revenues,ticker
0,910406,"THE HAIN CELESTIAL GROUP, INC.",2009-06-30,-2.472300e+07,1.122734e+09,HAIN
1,1357450,"HOLLYSYS AUTOMATION TECHNOLOGIES, LTD.",2009-06-30,-1.385106e+07,1.575021e+08,HOLI
2,7084,ARCHER-DANIELS-MIDLAND CO,2009-06-30,1.684000e+09,6.920700e+10,ADM
3,16058,CACI International Inc,2009-06-30,8.969800e+07,2.730162e+09,CACI
4,820318,II-VI INC,2009-06-30,3.678100e+07,2.922220e+08,IIVIP
...,...,...,...,...,...,...
23970,933034,STRATTEC SECURITY CORP,2022-07-03,7.032000e+06,4.522650e+08,STRT
23971,858877,"CISCO SYSTEMS, INC.",2022-07-30,1.181200e+10,5.155700e+10,CSCO
23972,1327567,"Palo Alto Networks, Inc",2022-07-31,-2.670000e+08,5.501500e+09,PANW
23973,746598,BRADY CORP,2022-07-31,1.499790e+08,1.302062e+09,BRC


In [89]:
df2=df
df2=df2.sort_values(by=['date']).reset_index(drop=True)
step=200
tickers=[]
ticker_subset=[]
prev_date = datetime.strptime(df2.loc[0,'date'],'%Y-%m-%d')

def add_ticker_prices_to_dataframe(df,tickers):
    # get data for ticker set and clean up into useful form
    ticker_df=pd.DataFrame(tickers,columns=['ticker','date'])
    end_date= datetime.strptime(ticker_df.loc[0,'date'],'%Y-%m-%d').date()
    raw_tickers=ticker_df.loc[:,'ticker'].tolist()
    df_tick = yf.download(raw_tickers, start = end_date - timedelta(days = 35), end = end_date, interval = "1mo", group_by='ticker')
    try:
        df_tick = df_tick.xs('Close', level=1, axis=1)
    except:
        df_tick = df_tick[['Close']]
        df_tick = df_tick.rename({'Close':tickers[0][0]},axis='columns')
        pass
    df_tick = df_tick.T.reset_index()
    df_tick=df_tick.rename({'index':'ticker'}, axis='columns')
    df_tick=df_tick.rename({df_tick.columns[1]:'close'}, axis='columns')
    for i in range(2,10):
        try:
            df_tick.columns[i]
            df_tick['close']= np.where(~df_tick['close'].isnull(), df_tick['close'],df_tick.iloc[:,i])
        except:
            pass
    df_tick=df_tick[['ticker','close']]
    df_tick=df_tick.merge(ticker_df, on=['ticker'], how='outer')
    df_tick=df_tick.dropna()

    # # now that price data is gotten and cleaned up, add to main df
    df=df.merge(df_tick, on=['ticker','date'], how='outer', suffixes=('', '_x'))
    if 'close_x' in df.columns:
        df['close']= np.where(~df['close'].isnull(), df['close'],df['close_x'])
        df=df.drop(['close_x'],axis='columns')
    return df

# step through dataframe and make list of lists of tickers close in date and within certain length
# so that the yfinance calls execute as quickly as possible and don't time out with too many requests
for index, value in enumerate(df2['ticker']):
    # get difference in date between this date and list start date
    current_date_string=df2.loc[index,'date']
    current_date = datetime.strptime(current_date_string,'%Y-%m-%d')
    diff = (current_date.date()-prev_date.date()).days
    # check if the diference is days is greater than 1 month
    # check if the number of fetches in the array is greater than the desired step
    if diff>30 or len(ticker_subset)>=step:
        tickers.append(ticker_subset)
        prev_date = current_date
        df2=add_ticker_prices_to_dataframe(df2,ticker_subset)
        ticker_subset=[]
        
    # add ticker to sub-list
    ticker_subset.append((value,current_date_string))
# handle leftover tickers
if len(ticker_subset):
    df2=add_ticker_prices_to_dataframe(df2,ticker_subset)

df2=df2.sort_values(by=['date']).reset_index(drop=True)
df2.to_csv('edgaryfinance.csv')
df3=pd.read_csv('edgaryfinance.csv', index_col= 0)
df3

[*********************100%***********************]  13 of 13 completed

1 Failed download:
- IIVIP: Data doesn't exist for startDate = 1243321200, endDate = 1246345200
[*********************100%***********************]  3 of 3 completed

1 Failed download:
- GDOT: Data doesn't exist for startDate = 1245999600, endDate = 1249023600
[*********************100%***********************]  27 of 27 completed

3 Failed downloads:
- RJF-PB: No data found for this date range, symbol may be delisted
- RJF-PA: No data found for this date range, symbol may be delisted
- CEQP-P: Data doesn't exist for startDate = 1248678000, endDate = 1251702000
[*********************100%***********************]  7 of 7 completed

1 Failed download:
- HOVVB: Data doesn't exist for startDate = 1251442800, endDate = 1254466800
[*********************100%***********************]  17 of 17 completed

1 Failed download:
- CHEF: Data doesn't exist for startDate = 1256281200, endDate = 1259308800
[*********************100%**

,cik,entityName,date,NetIncomeLoss,Revenues,ticker,close
0,910406,"THE HAIN CELESTIAL GROUP, INC.",2009-06-30,-2.472300e+07,1.122734e+09,HAIN,7.805000
1,779152,HENRY JACK & ASSOCIATES INC,2009-06-30,1.031020e+08,7.455930e+08,JKHY,20.750000
2,7084,ARCHER-DANIELS-MIDLAND CO,2009-06-30,1.684000e+09,6.920700e+10,ADM,26.770000
3,16058,CACI International Inc,2009-06-30,8.969800e+07,2.730162e+09,CACI,42.709999
4,820318,II-VI INC,2009-06-30,3.678100e+07,2.922220e+08,IIVIP,NaN
...,...,...,...,...,...,...,...
23970,933034,STRATTEC SECURITY CORP,2022-07-03,7.032000e+06,4.522650e+08,STRT,35.419998
23971,858877,"CISCO SYSTEMS, INC.",2022-07-30,1.181200e+10,5.155700e+10,CSCO,45.049999
23972,1327567,"Palo Alto Networks, Inc",2022-07-31,-2.670000e+08,5.501500e+09,PANW,502.779999
23973,746598,BRADY CORP,2022-07-31,1.499790e+08,1.302062e+09,BRC,48.509998


In [500]:
df2.to_csv('edgaryfinance.csv')
df3=pd.read_csv('edgaryfinance.csv', index_col= 0)
df3

,cik,entityName,date,NetIncomeLoss,Revenues,ticker,close
0,1041514,"Net 1 UEPS Technologies, Inc.",2019-06-30,-311007000,1.606350e+08,LSAK,4.050000
1,319201,KLA CORPORATION,2019-06-30,1175617000,4.568904e+09,KLAC,136.320007
2,1375365,"SUPER MICRO COMPUTER, INC.",2019-06-30,71918000,3.500360e+09,SMCI,18.309999
3,310354,STANDEX INTERNATIONAL CORPORATION,2019-06-30,67914000,6.399310e+08,SXI,70.370003
4,1606757,"KIMBALL ELECTRONICS, INC.",2019-06-30,31558000,1.181844e+09,KE,15.900000
...,...,...,...,...,...,...,...
2877,933034,STRATTEC SECURITY CORP,2020-06-28,-7605000,3.853000e+08,STRT,12.900000
2878,100726,"UNIFI, INC.",2020-06-28,-57237000,6.065090e+08,UFI,13.730000
2879,707549,LAM RESEARCH CORPORATION,2020-06-28,2251753000,1.004474e+10,LRCX,273.670013
2880,1084869,1 800 FLOWERS COM INC,2020-06-28,58998000,1.489637e+09,FLWS,22.139999


In [92]:
df4=df3.sort_values(by=['ticker','date'])
df4.to_csv('2009-2022NetIncomeRevenueAndPrice.csv')
df4.tail(50)

,cik,entityName,date,NetIncomeLoss,Revenues,ticker,close
14852,1585521,"Zoom Video Communications, Inc.",2019-01-31,7.584000e+06,3.305170e+08,ZM,NaN
17700,1585521,"Zoom Video Communications, Inc.",2020-01-31,2.530500e+07,6.226580e+08,ZM,68.040001
20657,1585521,"Zoom Video Communications, Inc.",2021-01-31,6.723160e+08,2.651368e+09,ZM,337.320007
23724,1585521,"Zoom Video Communications, Inc.",2022-01-31,1.375639e+09,4.099864e+09,ZM,211.410004
21370,1838937,Zhangmen Education Inc.,2021-12-31,-1.847170e+08,6.911190e+08,ZMENY,1.250000
9990,1713683,"ZSCALER, INC.",2017-07-31,-3.546000e+07,1.257170e+08,ZS,NaN
12506,1713683,"ZSCALER, INC.",2018-07-31,-3.364600e+07,1.901740e+08,ZS,26.260000
15129,1713683,"ZSCALER, INC.",2019-07-31,-2.865500e+07,3.028360e+08,ZS,76.639999
18015,1713683,"ZSCALER, INC.",2020-07-31,-1.151160e+08,4.312690e+08,ZS,98.089996
20949,1713683,"ZSCALER, INC.",2021-07-31,-2.620290e+08,6.731000e+08,ZS,194.199997
